In [1]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import pickle
import numpy as np

#load xgb model
xgb_model = XGBClassifier()
xgb_model.load_model(r"C:\Users\miles\favela_analysis\output\faz_quem_quer\faz_quem_quer_xgb_model.json")


In [2]:
# reconstruct label_encoder
loaded_classes = np.load(r"C:\Users\miles\favela_analysis\output\faz_quem_quer\faz_quem_quer_label_encoder_classes.npy")

# Recreate the LabelEncoder and assign the loaded classes_
label_encoder = LabelEncoder()
label_encoder.classes_ = loaded_classes  # For correct class mapping


In [5]:
# load neccessary data
with open(r"C:\Users\miles\favela_analysis\output\morro_da_guaiba\morro_da_guaiba_X_test.pkl", "rb") as f:
    X_test = pickle.load(f)

y_test = np.load(r"C:\Users\miles\favela_analysis\output\morro_da_guaiba\morro_da_guaiba_y_test.npy")


In [6]:
y_pred = xgb_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred) * 100
class_report_dict = classification_report(y_test, y_pred, target_names=label_encoder.classes_, output_dict=True)

print("Accuracy: ", accuracy)
print(class_report_dict)


Accuracy:  62.35997012696042
{'favela': {'precision': 0.6763990267639902, 'recall': 0.8881789137380192, 'f1-score': 0.7679558011049724, 'support': 939.0}, 'not-favela': {'precision': 0.009433962264150943, 'recall': 0.0025, 'f1-score': 0.003952569169960474, 'support': 400.0}, 'accuracy': 0.6235997012696042, 'macro avg': {'precision': 0.3429164945140706, 'recall': 0.44533945686900955, 'f1-score': 0.38595418513746643, 'support': 1339.0}, 'weighted avg': {'precision': 0.47715628904932583, 'recall': 0.6235997012696042, 'f1-score': 0.5397248132229674, 'support': 1339.0}}
